<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Next-actions-and-TODOs" data-toc-modified-id="Next-actions-and-TODOs-1"><strong>Next actions and TODOs</strong></a></span></li><li><span><a href="#Параметры-для-запуска" data-toc-modified-id="Параметры-для-запуска-2">Параметры для запуска</a></span></li><li><span><a href="#Считывание-изображения" data-toc-modified-id="Считывание-изображения-3">Считывание изображения</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#clahe" data-toc-modified-id="clahe-3.0.1">clahe</a></span></li></ul></li></ul></li><li><span><a href="#Фильтрация-изображения" data-toc-modified-id="Фильтрация-изображения-4">Фильтрация изображения</a></span></li><li><span><a href="#Определение-центра" data-toc-modified-id="Определение-центра-5">Определение центра</a></span></li><li><span><a href="#Выделение-сомы" data-toc-modified-id="Выделение-сомы-6">Выделение сомы</a></span></li><li><span><a href="#Матрица-Гессе" data-toc-modified-id="Матрица-Гессе-7">Матрица Гессе</a></span></li><li><span><a href="#Построение-графа" data-toc-modified-id="Построение-графа-8">Построение графа</a></span><ul class="toc-item"><li><span><a href="#Добавление-точек-сомы-в-граф" data-toc-modified-id="Добавление-точек-сомы-в-граф-8.1">Добавление точек сомы в граф</a></span></li></ul></li><li><span><a href="#Расчет-путей" data-toc-modified-id="Расчет-путей-9">Расчет путей</a></span></li><li><span><a href="#Расчет-встречаемости-и-фильтрация-точек-по-встречаемости" data-toc-modified-id="Расчет-встречаемости-и-фильтрация-точек-по-встречаемости-10">Расчет встречаемости и фильтрация точек по встречаемости</a></span><ul class="toc-item"><li><span><a href="#Ниже-ничего-особо-не-менялось" data-toc-modified-id="Ниже-ничего-особо-не-менялось-10.1">Ниже ничего особо не менялось</a></span></li></ul></li><li><span><a href="#Выделение-тонких-веток" data-toc-modified-id="Выделение-тонких-веток-11">Выделение тонких веток</a></span></li><li><span><a href="#Объединение" data-toc-modified-id="Объединение-12">Объединение</a></span><ul class="toc-item"><li><span><a href="#Выделение-крайних-точек" data-toc-modified-id="Выделение-крайних-точек-12.1">Выделение крайних точек</a></span></li><li><span><a href="#Ближайшие-точки-скелета" data-toc-modified-id="Ближайшие-точки-скелета-12.2">Ближайшие точки скелета</a></span></li><li><span><a href="#Добавление-соединений" data-toc-modified-id="Добавление-соединений-12.3">Добавление соединений</a></span></li></ul></li><li><span><a href="#Многомасштабный-гессиан" data-toc-modified-id="Многомасштабный-гессиан-13">Многомасштабный гессиан</a></span></li></ul></div>

# **Next actions and TODOs**
 - [ ] Test performance on other cells
 - [ ] Test performace of the approach with more sigma steps (log scale is preferred, i.e. `2.0**np.arange(-1,5,0.5)`)
 - [ ] Think about a way to regularize vector orientations, using orientations of the neighbours, or at different scales
 - [ ] Find a best way to skeletonize the qstack-based arrays and masks (as one of the approaches)
 - [ ] Find a way to "glue" together paths, that a close-by and have a similar direction

In [1]:
import os
import sys

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt

In [3]:
import cv2

In [4]:
#import cv2
import scipy
from scipy import ndimage as ndi
import numpy as np
import networkx as nx

from pathlib import Path

In [5]:
import napari

In [6]:
from tqdm.auto import tqdm

In [7]:
import ccdb
import astromorpho as astro

Can't load imreg package, affine and homography registrations won't work


/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/imfun/external/tifffile.py:7261: UserWarning: No module named 'imfun.external._tifffile'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/levtg/astro-morpho/venv/lib64/python3.10/site-packages/imfun/external/czifile.py:146: UserWarning: ImportError: No module named '_czifile'. Decoding of JXR and JPEG encoded images will not be available. Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  warnings.warn(
2022-07-21 13:39:21.923105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/levtg/astro-morpho/venv/lib/python3.10/site-packages/cv2/../../lib64:/home/levtg/v3d_external/v3d_main/common_lib/lib/
2022-07-21 13:39:21.923130: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dler

In [ ]:
from networx2napari import draw_edges, draw_nodes

In [ ]:
def weight_of_arr(a, b):
    dprod = np.einsum('...ij,...ij->...i', a, b)
    norm_a = np.linalg.norm(a, axis=-1)
    norm_b = np.linalg.norm(b, axis=-1)
    cos_dist = np.nan_to_num(dprod / ((norm_a * norm_b)), nan=0)
    cos_dist = 1 - cos_dist
    return np.abs(1 - cos_dist) * np.mean([norm_a, norm_b], axis=0)

In [ ]:
def calc_edges(arr1, arr2, index1, index2):
    weights = weight_of_arr(arr1, arr2)
    weight = weights.ravel()
    li = threshold_li(weight)
    idx1 = [tuple(i) for i in index1.reshape((-1, index1.shape[-1]))[weight>li]]
    idx2 = [tuple(i) for i in index2.reshape((-1, index2.shape[-1]))[weight>li]]
    return zip(idx1, idx2, 1 - weight[weight>li])

In [ ]:
from collections import defaultdict

def count_points_paths(paths):
    acc = defaultdict(int)
    for path in paths:
        for n in path:
            acc[n] += 1
    return acc

In [ ]:
from skimage.filters import threshold_li, threshold_minimum
from skimage.morphology import remove_small_objects

In [ ]:
# def remove_small_objects(mask, min_size=25):
#     labels, nlab = ndi.label(mask)
#     objs = ndi.find_objects(labels)
#     out_mask = np.zeros_like(mask)
#     for k,o in enumerate(objs):
#         cond = labels[o]==(k+1)
#         if np.sum(cond) >= min_size:
#             out_mask[o][cond] = True
#     return out_mask

def largest_region(mask):
    labels, nlab = ndi.label(mask)
    if nlab > 0:
        objs = ndi.find_objects(labels)
        sizes = [np.sum(labels[o]==k+1) for k,o in enumerate(objs)]
        k = np.argmax(sizes)
        return labels==k+1
    else:
        return mask
        
def crop_image(img, mask=None, margin=0, min_obj_size=0):
    if mask is None:
        mask = img > 0
    if min_obj_size > 0:
        mask = remove_small_objects(mask, min_obj_size)
    if margin > 0:
        mask = ndi.binary_dilation(mask, iterations=margin)
    objs = ndi.find_objects(mask)
    min_bnds = np.min([[sl.start for sl in o] for o in objs],0)
    max_bnds = np.max([[sl.stop for sl in o] for o in objs],0)
    crop = tuple(slice(mn,mx) for mn,mx in zip(min_bnds, max_bnds))
    return img[crop]

In [ ]:
plt.rc('figure', dpi=150)

# Параметры для запуска

In [ ]:
#filename = '3wk-both1-grn-raw.pic'
data_dir = '/home/levtg/astro-morpho/data/'
# data_dir = '/home/brazhe/yadisk/data-shared-comfi/3D-astrocyte-images/selected-for-complexity/'
filename = '3wk-both1-grn-raw.pic'

verbose = True
sigma = 2

# Set false to start from console
HANDY = True

# Set true to save output
OUT = False

In [ ]:
# Parameters
filename = "/home/levtg/astro-morpho/data/1wk-both1-red-raw.pic"
sigma = 1
HANDY = False


In [ ]:
filename = Path(data_dir).joinpath(filename)
filename

# Считывание изображения

In [ ]:
# if HANDY:
#     filename = '/home/levtg/astro-morpho/data/3wk-ly10-raw.pic'

In [ ]:
stack, meta = ccdb.read_pic(filename)
dims = ccdb.get_axes(meta)
dims

In [ ]:
if len(dims):
    zoom = (dims[-1][0]/dims[0][0])
else:
    zoom = 4
    
print(zoom)

### clahe

In [ ]:
clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))

In [ ]:
stack_shape = stack.shape
#stack_roll = np.reshape(stack, (stack_shape[0],-1))
#print("Rolled shape: ", stack_roll.shape)
#%time img = clahe.apply(stack_roll).reshape(stack.shape)

img = np.zeros(stack.shape, np.float32)
for k,plane in enumerate(stack):
    img[k] = clahe.apply(plane)

In [ ]:
if verbose:
    wi = napari.view_image(stack, ndisplay=3, scale=(zoom, 1,1), name='raw', colormap='cyan')
    wi.add_image(img, scale=(zoom,1,1), name='CLAHE',colormap='magenta')

In [ ]:
threshold_li(img)

In [ ]:
max_proj = img.max(0)

In [ ]:
domain_mask = ndi.binary_dilation(largest_region(remove_small_objects(max_proj > 0.5*threshold_li(max_proj))), iterations=3)
domain_mask = ndi.binary_closing(domain_mask,iterations=3)

In [ ]:
plt.imshow(max_proj, cmap='gray')
plt.contour(domain_mask, colors=['r'], levels=[0.5])

In [ ]:
img_cropped = np.array([crop_image(plane,domain_mask, margin=10) for plane in img])

In [ ]:
max_proj_1 = img_cropped.max(1)
domain_mask_1 = ndi.binary_dilation(largest_region(remove_small_objects(max_proj_1 > 0.5*threshold_li(max_proj_1))), iterations=3)
domain_mask_1 = ndi.binary_closing(domain_mask_1,iterations=3)
plt.imshow(max_proj_1, cmap='gray')
plt.contour(domain_mask_1, colors=['r'], levels=[0.5])

In [ ]:
img_cropped = np.array([crop_image(img_cropped[:,i],domain_mask_1, margin=10) for i in range(img_cropped.shape[1])]).swapaxes(0,1)

In [ ]:
if verbose:
    napari.view_image(img_cropped)

Важный вопрос, как сделать одинаковым масштаб по осям z и xy. Можно downsample XY, можно upsample (by interpolation) Z. Можно комбинировать. В идеале, наверное, XY не трогать, а сделть upsample по Z. 

In [ ]:
downscale = 2
%time img_noisy = ndi.zoom(img_cropped.astype(np.float32), (zoom/downscale, 1/downscale, 1/downscale), order=1)

In [ ]:
plt.imshow(img_noisy.max(0), cmap='gray')

In [ ]:
img.shape, img_noisy.shape

# Фильтрация изображения

In [ ]:
def filter_image(image, filter_func):
    threshold = filter_func(image)
    #img_filt = np.where(image > threshold, image, 0)
    binary_clean = remove_small_objects(image >= threshold, 5, connectivity=3)
    return np.where(binary_clean, image, 0)

In [ ]:
img_clear = filter_image(img_noisy, threshold_li)
if HANDY:
    pass
#     img_clear = filter_image(img_noisy, threshold_minimum)

In [ ]:
final_image = img_clear

In [ ]:
final_image.shape

In [ ]:
# НЕ ЗАКРЫВАТЬ!!!
# allW = napari.view_image(final_image, colormap='gray', ndisplay=3, opacity=0.5)

# Определение центра

In [ ]:
import itertools as itt

In [ ]:
def percentile_rescale(arr, plow=1, phigh=99.5):
    low, high = np.percentile(arr, (plow, phigh))
    if low == high:
        return np.zeros_like(arr)
    else:
        return np.clip((arr-low)/(high-low), 0, 1)

In [ ]:
def flat_indices(shape):
    idx = np.indices(shape)
    return np.hstack([np.ravel(x_)[:,None] for x_ in idx])

In [ ]:
X1a = flat_indices(final_image.shape)

In [ ]:
%time weights_s = percentile_rescale(np.ravel(ndi.gaussian_filter(final_image,5))**2,plow=99.5,phigh=99.99)

In [ ]:
center = tuple(map(int, np.sum(X1a*weights_s[:,None],axis=0)/np.sum(weights_s)))
center

# Выделение сомы

In [ ]:
from skimage.morphology import dilation, skeletonize, flood

In [ ]:
from astromorpho import morpho

**Альтернативный подход к сегментации сомы**
1. Работаем со сглаженным стеком
2. делаем первичную маску как flood из центра с толерантностью в 10% разницы между максимальным и минимальным значениями в стеке
3. Разрастаем (аналог flood) первичную маску в несколько итераций

In [ ]:
#soma_mask = largest_region(np.where(dilation(eroded), True, False))
#soma_mask = largest_region(final_image >= np.percentile(final_image, 99))

smooth_stack = ndi.gaussian_filter(final_image, 3)
tol = (smooth_stack.max() - smooth_stack[final_image>0].min())/10

print('tol:',tol)
%time soma_seed_mask = flood(smooth_stack, center, tolerance=tol)

In [ ]:
%time soma_mask = morpho.expand_mask(soma_seed_mask, smooth_stack, iterations = 10)

In [ ]:
if verbose:
    w = napari.view_image(final_image, ndisplay=3, opacity=0.5)
    w.add_image(soma_seed_mask, blending='additive', colormap='cyan')
    w.add_image(soma_mask, blending='additive', colormap='magenta')

In [ ]:
%time soma_all  = morpho.mask2points(soma_mask)
%time soma_shell = morpho.inside_boundary_pixels(soma_mask)

# Матрица Гессе

In [ ]:
sigmas = 2.0**np.arange(-1, 4, 1)
sigmas

In [ ]:
qstacks = {}

In [ ]:
sato_coll = {}
Vf_coll = {}

In [ ]:
for sigma in tqdm(sigmas):
    #astro.morpho.sato3d is newer and uses tensorflow (if it's installed)
    #optimally, the two variants of sato3d should be merged
    sato, Vf = astro.morpho.sato3d(final_image, sigma, hessian_variant='gradient_of_smoothed', do_brightness_correction=False, return_vectors=True)
    sato_coll[sigma] = (sato*sigma**2)*(final_image > 0)
    #sato_coll[sigma] = final_image*sato*sigma**2
    #sato_coll[sigma] = sato*(final_image>0)
    Vf_coll[sigma] = Vf[...,0][...,::-1]

In [ ]:
sato_coll.keys()

In [ ]:
lengths_coll = {sigma: astro.enh.percentile_rescale(sato)**0.5 for sigma, sato in sato_coll.items()}

In [ ]:
vectors_coll = {}

In [ ]:
for sigma in Vf_coll:
    Vfx = Vf_coll[sigma]
    V = Vfx[..., 0]
    U = Vfx[..., 1]
    C = Vfx[..., 2]
    lengths = lengths_coll[sigma]
    vectors_coll[sigma] = np.stack((U*lengths, V*lengths, C*lengths), axis=3)

In [ ]:
sato_best = np.argmax([sato_coll[sigma] for sigma in sigmas], axis=0)

In [ ]:
from ucats import masks as umasks

In [ ]:
ksigma = len(sigmas)-1
# largest_sigma = sigmas[ksigma]
# largest_sigma

In [ ]:
# largest_sigma_seed = largest_region(umasks.select_overlapping(sato_best == ksigma, soma_mask))
sato = sato_coll[sigmas[1]]#*(final_image)
#tol = (sato.max()-sato.min())/5
#print('tol:', tol)
#largest_sigma_seed2 =  largest_region(sato >= 0.5*np.mean(sato[soma_mask]))
threshold = threshold_li(sato[sato>0])
mask = remove_small_objects(sato>threshold, int(sigma*64))
#largest_sigma_seed = flood(sato, center, tolerance=tol)
# np.any(largest_sigma_seed)

In [ ]:
masks = {}
for sigma in tqdm(sigmas):
    sato = sato_coll[sigma]
    threshold = threshold_li(sato[sato>0])*sigma**0.5
    print(sigma, threshold)
    masks[sigma] = remove_small_objects(sato > threshold, min_size=int(sigma*64))

In [ ]:
masks[sigmas[-1]] = umasks.select_overlapping(masks[sigmas[-1]], soma_mask)

In [ ]:
for k in range(len(sigmas)-2,-1,-1):
    sigma = sigmas[k]
    masks[sigma] = umasks.select_overlapping(masks[sigma], ndi.binary_dilation(masks[sigmas[k+1]], iterations=5))

In [ ]:
if verbose:
    w = napari.view_image(final_image, )
    for sigma in masks:
        w.add_image(masks[sigma], blending='additive', name=f'σ={sigma:02f}')

In [ ]:
vectors_best = np.zeros(vectors_coll[sigmas[0]].shape)
for k in range(len(sigmas)-1,-1,-1):
    sigma = sigmas[k]
    mask = masks[sigma]
    if k < len(sigmas)-1:
        mask = mask & (mask ^ masks[sigmas[k+1]])
    vectors_best[mask] = vectors_coll[sigma][mask]

In [ ]:
from hessian_vectors import add_hessian_vectors, sato2napari_vectors

In [ ]:
def vectors2napari(vectors, index=1):
    nr, nc, nd = vectors.shape[:-1]
    indexgrid = np.meshgrid(np.arange(nc), np.arange(nr), np.arange(nd))
    
#     length = np.ravel(np.linalg.norm(vectors, axis=-1))

    x, y, z = [np.ravel(a)[::index] for a in indexgrid]
    x1, y1, z1 = np.ravel(vectors[...,0])[::index], np.ravel(vectors[...,1])[::index], np.ravel(vectors[...,2])[::index]

    
    
    vectors = np.zeros((*x1.shape, 2, 3))
    vectors[..., 0, 0] = y
    vectors[..., 0, 1] = x
    vectors[..., 1, 0] = y1
    vectors[..., 1, 1] = x1

    vectors[..., 0, 2] = z
    vectors[..., 1, 2] = z1
    
    return vectors

In [ ]:
# vb = napari.view_image(final_image)
# add_hessian_vectors(vb, vectors2napari(vectors_best, index=50), np.ravel(sato_best)[::50])

# Построение графа

In [ ]:
def prep_crops():
    "makes list of crops for edges"
    num2slice = {1: (slice(1,None), slice(None,-1)), 
                 0: (slice(None), slice(None)), 
                -1: (slice(None,-1), slice(1,None))}
    shifts = list(itt.product(*[(-1,0,1)]*3))
    # we only need one half of that
    cut = int(np.ceil(len(shifts)/2))
    crops_new = [list(zip(*[num2slice[n] for n in tuple])) for tuple in shifts[cut:]]
    return crops_new

In [ ]:
crops_new = prep_crops()

In [ ]:
graph_coll = {sigma:nx.Graph() for sigma in sigmas}
nodes_coll = {sigma:{} for sigma in sigmas}

In [ ]:
graph_coll['best'] = nx.Graph()
nodes_coll['best'] = {}

In [ ]:
i, j, k = np.indices(final_image.shape)
idx = np.stack((i,j,k), axis=3)
idx.shape

In [ ]:
for sigma in sigmas:
    vectors = vectors_coll[sigma]#*(largest_sigma_seed[...,None])
    vectors = vectors*masks[sigma][...,None]
    #if sigma == largest_sigma:
    #    vectors = vectors*largest_sigma_seed[...,None]
    for crop, acrop in tqdm(crops_new):
         graph_coll[sigma].add_weighted_edges_from(calc_edges(vectors[crop], vectors[acrop], idx[crop], idx[acrop]))

In [ ]:
key='best'
vectors = vectors_best
graph_coll[key] = nx.Graph()
for crop, acrop in tqdm(crops_new):
         graph_coll[key].add_weighted_edges_from(calc_edges(vectors[crop], vectors[acrop], idx[crop], idx[acrop]))

## Добавление точек сомы в граф

In [ ]:
def get_mask_vals(idxs, mask):
    idx_mask = mask[idxs[:,0], idxs[:,1], idxs[:,2]]
    return idxs[idx_mask]

In [ ]:
def get_edges(mask, index1, index2, weight):
    idx1 = [tuple(i) for i in get_mask_vals(index1.reshape((-1, index1.shape[-1])), mask)]
    idx2 = [tuple(i) for i in get_mask_vals(index2.reshape((-1, index2.shape[-1])), mask)]
    return zip(idx1, idx2, np.full(len(idx1), weight))

In [ ]:
Gsoma = nx.Graph()

In [ ]:
for crop, acrop in tqdm(crops_new):
    Gsoma.add_weighted_edges_from(get_edges(soma_mask, idx[crop], idx[acrop], 0.7))

In [ ]:
# nx.set_edge_attributes(G, 0.7, name='weight')
# nx.set_edge_attributes(G, 'soma', name='area')

In [ ]:
graph_coll

In [ ]:
%%time 

#G = graph_coll[largest_sigma]
#G = graph_coll[current_sigma]

for key in graph_coll:
    G = graph_coll[key]
    for p1, p2, weight in Gsoma.edges(data=True):
        try:
            old_weight = G.get_edge_data(p1, p2)['weight']
        except Exception as exc:
            old_weight = 1
        G.add_edge(p1, p2, weight=min(weight['weight'], old_weight))

In [ ]:
nodes_coll = {key:{n:n for n in G.nodes()} for key, G in graph_coll.items()} # just a copy of G3 nodes

# Расчет путей

In [ ]:
%%time 

# fix negative weights

for G in graph_coll.values():
#     edges = [(u,v,d) for u,v,d in G.edges(data=True) if d['weight']<0]
    for p1,p2, weight in tqdm(G.edges(data=True)):
        if weight['weight'] < 0:
            print(p1,p2,weight)
            G.add_edge(p1,p2, weight=0)

In [ ]:
%%time 

# fix negative weights

for G in tqdm(graph_coll.values()):
    edges = {(u,v): 0 for u,v,d in G.edges(data=True) if d['weight']<0}
    nx.set_edge_attributes(G, edges, 'weight')

In [ ]:
len(soma_shell)

# Расчет встречаемости и фильтрация точек по встречаемости

In [ ]:
from copy import copy

def make_qstack(G, targets, min_count=1):
    paths_dict = nx.multi_source_dijkstra_path(G, targets, )
    paths = list(paths_dict.values())
    points = count_points_paths(paths)
    #to_del = [point for point,count in points.items() if count <=1]
    #points_count = copy(points)
    #for point in to_del:
    #    del points_count[point]
    qstack = np.zeros(vectors.shape[:-1])  #Это встречаемость точек в путях
    for p, val in points.items():
        if val >= min_count:
            qstack[p] = np.log(val)
    return qstack, paths

In [ ]:
#qstacks  = {sigma:make_qstack(graph_coll[sigma], soma_shell)[0] for sigma in tqdm(graph_coll)}

In [ ]:
qstack_masks = {}

In [ ]:
sigmas

In [ ]:
for ksigma in range(len(sigmas)-1,-1,-1):
    #alternative: use shell of a full mask as target
    sigma = sigmas[ksigma]
    print(ksigma, sigma)
    G = graph_coll[sigma]
    
    if ksigma == len(sigmas)-1:
        targets = soma_shell
    else:
        higher_sigma = sigmas[ksigma+1]        
        #targets = astro.morpho.boundary_pixels(masks[higher_sigma])
        targets = astro.morpho.mask2points(qstack_masks[higher_sigma])
        targets = [t for t in targets if G.has_node(t)]
        if not len(targets):
            print('No points from targets in the graph')
            continue
    qstack = make_qstack(G, targets)[0]
    qstack_mask = qstack > threshold_li(qstack[qstack>0])
    if not np.any(qstack_mask):
        print('empty qstack mask!')
        if ksigma < len(sigmas)-1:
            qstack_mask = qstack_masks[higher_sigma]
    qstacks[sigma] = qstack
    qstack_masks[sigma] = qstack_mask

In [ ]:
isinstance(2.0, (int, float))

Можно предложить как минимум, два варианта объединения масштабов:
 1. [ ] "Best" -- это где вектора в каждом вокселе взяты из соответствующих масок для разных масштабов, потом все это сведено в один граф, и во всем графе
         ищется путь до поверхности сомы. **NOTE:** по идее, маски должны быть "исключительными", то есть каждая область может принадлежать только одной сигме.
 2. [ ] "Combined" -- скелет и пути задаются итеративно от больших масштабов к маленьким, то есть используется свой граф для каждого масштаба и пути ищутся в дополнение к уже найденым. 
       Кстати, можно сделать лучше (предположительно), если вектора из qstack_mask старшего масштаба добавлять к графу меньшего масштаба и опять искать пути до сомы. Тогда будут дополнительно 
       "тренироваться" пути вдоль больших веток. 
       Потом можно брать просто сумму qstacks для разных масштабов, маску можно брать как объединение всех масок на разных уровнях или снова как надпороговые пиксели. 

In [ ]:
qstacks['best'] = make_qstack(graph_coll['best'], soma_shell)[0]
qstack_masks['best'] = qstacks['best'] > threshold_li(qstacks['best'][qstacks['best']>0])


In [ ]:
qstacks['combined'] = sum(qstacks[sigma] for sigma in sigmas)
qstack_masks['combined'] = sum(qstack_masks[sigma] for sigma in sigmas) > 0

In [ ]:
#sigma = sigmas[ksigma]
#sigma = largest_sigma

if verbose:
    #w.add_image(qstack)
    w = napari.view_image(final_image, ndisplay=3, opacity=0.5)
    w.add_image(soma_mask, name='soma', interpolation='nearest', blending='additive',colormap='cyan')
    #w.add_image(sato_coll[current_sigma], name=f'sato@{current_sigma:0.2f}', colormap='inferno', blending='additive')
    #w.add_image(largest_sigma_seed.astype(np.float32), name=f'largest_sigma_seed', interpolation='nearest', colormap='blue', blending='additive')
    for sigma in qstacks:
        name = sigma if isinstance(sigma, str) else f'qstack@{sigma:0.2f}'
        qstack = qstacks[sigma]
        data_range = np.percentile(qstack[qstack>0], (1,99))
        w.add_image(qstack, name=name, interpolation='nearest', blending='additive', contrast_limits=data_range, visible=False)
        #threshold = threshold_li(qstack[qstack>0])
        w.add_image(qstack_masks[sigma], name=name+':qmask', blending='additive', colormap='red')
        if isinstance(sigma, (int, float)):
            w.add_image(masks[sigma], name=f'mask@{sigma:0.2f}', blending='additive', colormap='green',visible=False)
    #w.add_image(qstacks[sigma], name=f'qstack@{sigma:0.2f}', interpolation='nearest', colormap='magenta', blending='additive',contrast_limits=(0,12))
    #w.add_image(sato_best == len(sigmas)-1, name='kbest', colormap='red',blending='additive')
    #w.add_image(qstacks[sigma]*largest_sigma_seed, name=f'x@{sigma:0.2f}', interpolation='nearest', colormap='green', blending='additive', contrast_limits=(0,12))
        

In [ ]:
combined_mask = sum(qstack_masks[sigma] for sigma in qstack_masks) > 0

In [ ]:
combined_mask = remove_small_objects(combined_mask, 4)

In [ ]:
combined_skeleton = skeletonize(combined_mask)

In [ ]:
w = napari.view_image(final_image, ndisplay=3, opacity=0.5)
w.add_image(combined_mask, colormap='green', blending='additive')
w.add_image(combined_skeleton, colormap='red', blending='additive')

In [ ]:
#qstacks[largest_sigma]

In [ ]:
qstacks[sigmas[-2]] = make_qstack(graph_coll[sigmas[-2]], astro.morpho.mask2points(qstacks[sigmas[-1]]>3.5))[0]

## Ниже ничего особо не менялось

In [ ]:
if OUT:
    np.save(f'{os.path.basename(filename)}-qstacks.npy',qstacks)

In [ ]:
qstack_mask = qstack > 1

In [ ]:
#napari.view_image(skeletonize(qstack_mask))
# = napari.view_image(final_image, ndisplay=3)
#w.add_image(qstack_mask)

In [ ]:
p = np.asarray(list(points_count.keys()))

In [ ]:
vals = np.asarray(list(points_count.values()))

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(np.log(vals[vals>1]), bins=500)
#plt.xlim(-10, 100)
plt.axvline(np.mean(np.log(vals)), color='cyan', ls='--', label='th.mean')
plt.axvline(threshold_li(np.log(vals)), color='tomato', ls='--', label='th.Li')
plt.axvline(np.log(np.mean(vals)), color='b', ls='--', label='th.log_mean')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(vals[vals>1], bins=500)
#plt.xlim(-10, 100)
plt.axvline(np.mean(vals), color='b', ls='--', label='th.mean')
plt.axvline(threshold_li(vals), color='tomato', ls='--', label='th.Li')
plt.axvline(np.exp(np.mean(np.log(vals))), color='cyan', ls='--', label='th.log_mean')
plt.legend()
plt.show()

In [ ]:
points_mean = [k for k, v in points_count.items() if v > np.mean(vals)]
thr_li = threshold_li(vals)
points_li = [k for k, v in points_count.items() if v > thr_li]
# points_log_mean = [k for k, v in points_count.items() if np.log(v) > np.mean(np.log(vals))]
# points_log_li = [k for k, v in points_count.items() if np.log(v) > threshold_li(np.log(vals))]

In [ ]:
if verbose:
    w.add_points(draw_nodes(nodes3, points_mean), size=1, edge_color='transparent', face_color='c', name='filt mean')
    w.add_points(draw_nodes(nodes3, points_li), size=1, edge_color='transparent', face_color='g', name='filt li')

# Выделение тонких веток

In [ ]:
Gsub = G3.subgraph(points_mean)
nodes_sub = {n:n for n in Gsub.nodes()}

In [ ]:
props = {'weight': 1 - np.array([edgedata["weight"] for _, _, edgedata in Gsub.edges(data=True)])}

In [ ]:
w.add_shapes(draw_edges(nodes_sub, list(Gsub.edges())), shape_type='path', edge_color='weight', edge_width=0.1, edge_colormap='inferno', properties=props)

In [ ]:

c = np.array(Gsub.nodes())

In [ ]:
graph_mask = np.full(final_image.shape, False)
graph_mask[c[:,0], c[:,1], c[:,2]] = True

In [ ]:
th_branch_mask = np.full(final_image.shape, False)
th_branch_mask[c[:,0], c[:,1], c[:,2]] = True
th_branch_mask[flood_mask] = False

In [ ]:
th_branch_mask = remove_small_objects(th_branch_mask, 5, connectivity=3)

# Объединение

In [ ]:
branch_mask = np.full(final_image.shape, False)
branch_mask[th_branch_mask] = True
branch_mask[w_branch_mask] = True
branch_mask[soma_mask] = True

if verbose:
    bm = napari.view_image(final_image, colormap='magenta', ndisplay=3)
    bm.add_image(branch_mask, blending='additive', colormap='cyan')

file:///home/levtg/astro-morpho/data/pictures_nb/join%20plan.jpg![image.png](attachment:image.png)

## Выделение крайних точек

In [ ]:
from skimage.morphology import cube
edge = dilation(flood_mask, cube(3)) ^ flood_mask
points = edge & th_branch_mask

In [ ]:
if verbose:
    bm.add_image(points, blending='additive', colormap='red', gamma=0.2)

In [ ]:
if verbose:
    bm.add_image(w_branch_mask, blending='additive')
    bm.add_image(th_branch_mask, blending='additive')
    bm.add_image(soma_mask, blending='additive')
    bm.add_image(flood_mask, opacity=0.5, colormap='red')

## Ближайшие точки скелета

In [ ]:
points_coords = idx[points]
skelet_coords = idx[w_branch_mask]

In [ ]:
points2connect = {}
for point in points_coords:
    point_dists = np.linalg.norm(point - skelet_coords, axis=-1)
    if len(points2connect.keys()) > 0:
        point_n_dists = np.linalg.norm(point - np.array(list(points2connect.keys())), axis=-1)
        if point_n_dists.min() < 1.5:
            continue
    if point_dists.min() > 1.5:
        coord = skelet_coords[point_dists.argmin()]
        points2connect[tuple(point)] = tuple(coord)

In [ ]:
print(points_coords.shape, len(points2connect.keys()))

## Добавление соединений

In [ ]:
# from networkx.algorithms.shortest_paths.weighted import multi_source_dijkstra
full_mask = branch_mask.copy()
for s, e in points2connect.items():
#     d, p = multi_source_dijkstra(G3, [tuple(n) for n in skelet_coords], s, 'weight')
    path = nx.shortest_path(G3, s, e, 'weight')
    path_coords = np.array(path)
    full_mask[path_coords[:,0], path_coords[:, 1], path_coords[:,2]] = True

In [ ]:
bm.add_image(full_mask)

# Многомасштабный гессиан

In [ ]:
from skimage.filters import ridges as skridges

![image.png](attachment:image.png)

https://content.iospress.com/download/bio-medical-materials-and-engineering/bme1149?id=bio-medical-materials-and-engineering%2Fbme1149


In [ ]:
def vesselness(img, sigma, alpha= 0.5, beta=0.5, cc=20):
    #H = skf.hessian_matrix(img,sigma)
    #Hl = skf.hessian_matrix_eigvals(H)
    Hl = skridges.compute_hessian_eigenvalues(img, sigma)
    sh = img.shape
    v = np.zeros(sh)
    for r in range(sh[0]):
        for c in range(sh[1]):
            for h in range(sh[2]):
                arr = (Hl[0][r,c,h], Hl[1][r,c,h], Hl[2][r,c,h])

                l1,l2,l3 = sorted(arr, key=abs)
                
                if l2 >= 0 or l3 >=0:
                    v[r,c,h] = 0
                else:
                    Ra = np.abs(l2/l3)
                    Rb = np.abs(l1/np.sqrt(np.abs(l2*l3)))
                    S = np.sqrt(l1**2 + l2**2 + l3**2)
                    v[r,c, h] =  (1 - np.exp(-Ra**2/(2*alpha**2)))*np.exp(-Rb**2/(2*beta**2))*(1 - np.exp(-S**2/(2*cc**2)))
    return v


def multiscale_vesselness(img, sigma_start, sigma_stop, nlevels=50, reduce_fn=np.max):
    return reduce_fn([vesselness(img, sigma)*sigma**2 for sigma in np.linspace(sigma_start,sigma_stop, nlevels)],0)


In [ ]:
def multiscale_sigma(img, sigma_start, sigma_stop, nlevels=50):
    sout = np.zeros(img.shape)
    hout = np.zeros(img.shape)
    vout = np.zeros((*img.shape, 3, 3))
    for sigma in tqdm(np.linspace(sigma_start, sigma_stop, nlevels)):
        hcurr, vcurr = astro.hessian.sato3d(img, sigma, hessian_variant='gradient_of_smoothed', return_vectors=True)
        hcurr *= sigma**2
        mask = hcurr > hout
        
        hout[mask] = hcurr[mask]
        sout[mask] = sigma
        vout[mask] = vcurr[mask]
    return hout, sout, vout

In [ ]:
h, s, v = multiscale_sigma(final_image, 0.5, 4)

In [ ]:
hs = napari.view_image(s, ndisplay=3)
hs.add_image(h)

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
colors = ["black", ""]
cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)

In [ ]:
hs.add_image(np.where(full_mask^soma_mask, s, 0))

In [ ]:
hs.add_image(final_image)

In [ ]:
from hessian_vectors import add_hessian_vectors, sato2napari_vectors

In [ ]:
add_hessian_vectors(hs, *sato2napari_vectors(h, v))

In [ ]:
v_n, h_n = sato2napari_vectors(h, v)
v_n.shape



In [ ]:
add_hessian_vectors(hs, v_n.reshape((*full_mask.shape,2,3))[full_mask], h_n.reshape(full_mask.shape)[full_mask])

In [ ]:
sigma_skel = np.where(full_mask, s, 0)
hs.add_image(np.where(sigma_skel < 1, sigma_skel, 0), blending='additive', name='0.5-1')
hs.add_image(np.where(np.where(sigma_skel > 1, sigma_skel, 0) < 2, sigma_skel, 0), blending='additive', name='1-2')
hs.add_image(np.where(np.where(sigma_skel > 2, sigma_skel, 0) < 3, sigma_skel, 0), blending='additive', name='2-3')
hs.add_image(np.where(sigma_skel > 3, sigma_skel, 0), blending='additive', name='3-4')

In [ ]:
w.add_image(full_mask)